# Transfer Learning - ResNet50 이용

### [Transfer Learning을 위한 준비]

_ResNet50의 input shape은 (224, 224, 3)인데 (52, 52, 3)로 바꿔도 될까? (By RandomCrop)_

transform.Normalize의 경우 input tensor의 정규화를 의미.
[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]는 ImageNet데이터의 평균과 표본편차 값.

In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy


USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

BATCH_SIZE = 256
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize([64, 64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './dataset'
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']}
data_loaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['train', 'val']}
dataset_size = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

### [Pre-Trained Model 불러오기]

In [8]:
from torchvision import models

resnet = models.resnet50(pretrained=True)
print(resnet)

/Users/cheongwonook/opt/anaconda3/envs/UsePyTorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/cheongwonook/opt/anaconda3/envs/UsePyTorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

(1) Difference of nn.CrossEntropyLoss(), nn.Functional.cross_entropy() <br><br>
같은 기능으로 작동. 단지 Layer와 Function의 차이만 있을 뿐이다. 개발하는 스타일의 차이. <br>
다만 Layer는 처음에 initiation을 해주어야 한다. <br><br>
_표기상 Layer와 Function임을 구분하기 위해 loss_layer, loss_fn으로 표기할 것임._

(2) Epoch에 따라 Learning_Rate를 변화시켜주는 메서드. <br><br>
step_size=7, gamma=0.1이면 7Epoch마다 lr에 0.01을 곱하겠다는 의미.

In [9]:
num_ftrs = resnet.fc.in_features # resnet의 fc layer의 input features
resnet.fc = nn.Linear(num_ftrs, 33) # fc Layer 수정
resnet = resnet.to(DEVICE)

loss_layer = nn.CrossEntropyLoss() # (1)

optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1) # (2)

### [Pre-Trained Model의 일부 Layer Freeze 하기]

In [10]:
ct = 0
for child in resnet.children():
    # print(child)
    ct += 1
    if ct < 6:
        for param in child.parameters():
            param.requires_grad = False

### [Transfer Learning 모델 학습과 검증을 위한 함수]

_(1) labels 와 labels.data 간의 차이?_

In [11]:
import time

def train_resnet(model, loss_layer, optimizer, scheduler, num_epochs=30):

    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(1, num_epochs+1):
        print('------------------ epoch {}------------------'.format(epoch))
        since = time.time()

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # torch.max operation는 max값과 index 값을 tuple형태로 반환
                    loss = loss_layer(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data) # (1)

            if phase == 'train':
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]
                print('learning_rate: ', l_r)
            
            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]
        
            print('{} Loss: {:.4f}, Accuracy: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    print('Best val Acc: {:.4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)

    return model


### [모델 학습 실행하기]

In [20]:
model_resnet50 = train_resnet(resnet, loss_layer, optimizer_ft, exp_lr_scheduler, 10)

torch.save(model_resnet50, 'resnet50.pt')

------------------ epoch 1------------------
learning_rate:  [0.001]
train Loss: 0.1228, Accuracy: 0.9597
val Loss: 0.1268, Accuracy: 0.9598
Completed in 9m 48s
------------------ epoch 2------------------
learning_rate:  [0.001]
train Loss: 0.1051, Accuracy: 0.9647
val Loss: 0.1260, Accuracy: 0.9601
Completed in 9m 11s
------------------ epoch 3------------------
learning_rate:  [0.001]
train Loss: 0.0944, Accuracy: 0.9699
val Loss: 0.1014, Accuracy: 0.9707
Completed in 9m 29s
------------------ epoch 4------------------
learning_rate:  [0.0001]
train Loss: 0.0895, Accuracy: 0.9706
val Loss: 0.1052, Accuracy: 0.9656
Completed in 9m 6s
------------------ epoch 5------------------
learning_rate:  [0.0001]
train Loss: 0.0455, Accuracy: 0.9848
val Loss: 0.0395, Accuracy: 0.9876
Completed in 11m 50s
------------------ epoch 6------------------
learning_rate:  [0.0001]
train Loss: 0.0283, Accuracy: 0.9909
val Loss: 0.0401, Accuracy: 0.9857
Completed in 7m 15s
------------------ epoch 7-----

## 모델 평가

### [Transfer Learning 모델 평가를 위한 전처리]

In [12]:
transform_resNet = transforms.Compose([transforms.Resize([64, 64]), 
                                       transforms.RandomCrop(52),
                                       transforms.ToTensor(), 
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_resNet = ImageFolder(root='./dataset/test', transform=transform_resNet)
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

### [Transfer Learning 모델 성능 평가하기]

In [13]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()

            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset) # test dataset의 전체 loss
    test_acc = 100 * correct / len(test_loader.dataset)
    return test_loss, test_acc

In [17]:
resnet50 = torch.load('resnet50.pt')
resnet50.eval()
test_loss, test_acc = evaluate(resnet50, test_loader_resNet)

print('baseline test acc: ', test_acc)


baseline test acc:  99.01520817751184
